In [1]:
import pandas as pd
import numpy as np
import os, sys

sys.path.append(os.path.abspath('..'))  # adds the parent directory

#from src.data_processing import TransactionFeatureExtractor, create_proxy_target
from src.train import train_model

print("All imports done.")

All imports done.


In [2]:
import mlflow
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [3]:
# Cell 2: Load and preprocess data
print("🔄 Loading data...")
df = pd.read_csv("..\\data\\processed\\data_cleaned.csv")

🔄 Loading data...


In [ ]:
df.head()

,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_year,total_transaction_amount,average_transaction_amount,transaction_count,std_transaction_amount,Recency,Frequency,Monetary,is_high_risk
0,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2,0,2,15,11,2018,109921.75,923.712185,119,3042.294251,1,119,109921.75,0
1,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2,0,2,15,11,2018,109921.75,923.712185,119,3042.294251,1,119,109921.75,0
2,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2,0,2,15,11,2018,1000.00,500.000000,2,0.000000,82,2,1000.00,1
3,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2,0,3,15,11,2018,228727.20,6019.136842,38,17169.241610,6,38,228727.20,0
4,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2,0,3,15,11,2018,228727.20,6019.136842,38,17169.241610,6,38,228727.20,0


In [ ]:
df.select_dtypes(exclude=['object']).columns

Index(['Amount', 'Value', 'PricingStrategy', 'FraudResult', 'transaction_hour',
       'transaction_day', 'transaction_month', 'transaction_year',
       'total_transaction_amount', 'average_transaction_amount',
       'transaction_count', 'std_transaction_amount', 'Recency', 'Frequency',
       'Monetary', 'is_high_risk'],
      dtype='object')

In [ ]:
df.head()

,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_year,total_transaction_amount,average_transaction_amount,transaction_count,std_transaction_amount,Recency,Frequency,Monetary,is_high_risk
0,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2,0,2,15,11,2018,109921.75,923.712185,119,3042.294251,1,119,109921.75,0
1,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2,0,2,15,11,2018,109921.75,923.712185,119,3042.294251,1,119,109921.75,0
2,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2,0,2,15,11,2018,1000.00,500.000000,2,0.000000,82,2,1000.00,1
3,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2,0,3,15,11,2018,228727.20,6019.136842,38,17169.241610,6,38,228727.20,0
4,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2,0,3,15,11,2018,228727.20,6019.136842,38,17169.241610,6,38,228727.20,0


In [17]:
df.dropna(inplace=True)

In [ ]:
df.columns

Index(['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'Amount',
       'Value', 'PricingStrategy', 'FraudResult', 'transaction_hour',
       'transaction_day', 'transaction_month', 'transaction_year',
       'total_transaction_amount', 'average_transaction_amount',
       'transaction_count', 'std_transaction_amount', 'Recency', 'Frequency',
       'Monetary', 'is_high_risk'],
      dtype='object')

In [18]:
numeric_cols = ['Amount', 'Value', 'PricingStrategy', 'FraudResult', 'transaction_hour',
       'transaction_day', 'transaction_month', 'transaction_year',
       'total_transaction_amount', 'average_transaction_amount',
       'transaction_count', 'std_transaction_amount', 'Recency', 'Frequency',
       'Monetary']

categorical_cols = ['ChannelId', 'ProductCategory', 'ProviderId']

In [19]:
y = df['is_high_risk']
X = df.drop(columns=['is_high_risk'])

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [21]:
X_train.columns

Index(['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'Amount',
       'Value', 'PricingStrategy', 'FraudResult', 'transaction_hour',
       'transaction_day', 'transaction_month', 'transaction_year',
       'total_transaction_amount', 'average_transaction_amount',
       'transaction_count', 'std_transaction_amount', 'Recency', 'Frequency',
       'Monetary'],
      dtype='object')

In [22]:
y_train.value_counts()

is_high_risk
0    63288
1     7924
Name: count, dtype: int64

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.pipeline import Pipeline as ImbPipeline  # Use this for sampling
from imblearn.over_sampling import SMOTE  # or RandomUnderSampler
from sklearn.linear_model import LogisticRegression  # Example model

# Define preprocessing
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
])

# Define the full pipeline with oversampling
pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('sampler', SMOTE(random_state=42)),  # Or RandomUnderSampler
    ('model', LogisticRegression(class_weight='balanced', max_iter=1000))
])

In [24]:
# === Define models to try ===
models = {
    'logistic_regression': LogisticRegression(class_weight='balanced', max_iter=1000),
    'decision_tree': DecisionTreeClassifier(),
    'random_forest': RandomForestClassifier()
}

In [14]:
# === Track best model ===
best_score = 0
best_model_name = None
best_pipeline = None

mlflow.set_experiment("credit-risk-model")

2025/07/01 21:23:17 INFO mlflow.tracking.fluent: Experiment with name 'credit-risk-model' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:///d:/Project/10 Academy Resourse/credit '
 'risk/credit-risk-model/mlruns/578058596288668658'), creation_time=1751394197263, experiment_id='578058596288668658', last_update_time=1751394197263, lifecycle_stage='active', name='credit-risk-model', tags={}>

In [15]:
# Set the correct tracking URI to the existing mlruns directory
mlflow.set_tracking_uri("..//mlruns")


In [26]:
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        # Build full pipeline
        pipeline = ImbPipeline([
            ('preprocessor', preprocessor),
            ('sampler', SMOTE(random_state=42)),
            ('model', model)
        ])

        # Fit model
        pipeline.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = pipeline.predict(X_test)
        y_proba = pipeline.predict_proba(X_test)[:, 1]
        f1 = f1_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_proba)

        # Log metrics and model
        mlflow.log_param("model_name", name)
        mlflow.log_metric("f1_score", f1)
        mlflow.log_metric("roc_auc", auc)
        mlflow.sklearn.log_model(pipeline, name="BestCreditRiskModel", registered_model_name="BestCreditRiskModel")

        print(f"✅ Trained {name} | F1: {f1:.4f} | AUC: {auc:.4f}")

        # Track best
        if f1 > best_score:
            best_score = f1
            best_model_name = name
            best_pipeline = pipeline

print(f"\n🏆 Best model: {best_model_name} with F1: {best_score:.4f}")

2025/07/01 21:28:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'BestCreditRiskModel'.
Created version '1' of model 'BestCreditRiskModel'.


✅ Trained logistic_regression | F1: 0.9962 | AUC: 1.0000


2025/07/01 21:29:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'BestCreditRiskModel' already exists. Creating a new version of this model...
Created version '2' of model 'BestCreditRiskModel'.


✅ Trained decision_tree | F1: 1.0000 | AUC: 1.0000


2025/07/01 21:30:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Trained random_forest | F1: 0.9998 | AUC: 1.0000

🏆 Best model: decision_tree with F1: 1.0000


Registered model 'BestCreditRiskModel' already exists. Creating a new version of this model...
Created version '3' of model 'BestCreditRiskModel'.


In [12]:
import mlflow.sklearn

# Load the best registered model from MLflow registry
model_uri = "models:/BestCreditRiskModel/6"  # or use version number instead of "Production"

loaded_pipeline = mlflow.sklearn.load_model(model_uri)

# Now you can use it to predict on new data
y_pred = loaded_pipeline.predict(X_test)
y_proba = loaded_pipeline.predict_proba(X_test)[:, 1]


MlflowException: Registered Model with name=BestCreditRiskModel not found

In [ ]:
for model in models:
    print(f"🔄 Training model: {model}")
    pipeline = Pipeline([
        ('model', model)
    ])

    pipeline.fit(X_resampled, y_resampled)
    y_pred = pipeline.predict(X_test_full)
    y_proba = pipeline.predict_proba(X_test_full)[:, 1]

    roc_auc = roc_auc_score(y_test, y_proba)
    f1 = f1_score(y_test, y_pred)

    print(f"✅ Model: {model}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(classification_report(y_test, y_pred))


In [136]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Amount', 'Value',
                                                   'PricingStrategy',
                                                   'FraudResult',
                                                   'transaction_hour',
                                                   'transaction_day',
                                                   'transaction_month',
                                                   'transaction_year',
                                                   'total_transaction_amount',
                                                   'average_transaction_amount',
                                                   'transaction_count',
                                                   'std_transaction_amount',
                                                   'Recency', 'Frequency',
                                                   'Monetary']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['ChannelId',
                                                   'ProductCategory',
                                                   'ProviderId'])])),
                ('sampler', SMOTE(random_state=42)),
                ('model',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [137]:
# Predict on test data
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

In [138]:
roc_auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)

print(f'roc : {roc_auc}')
print(f'f1 : {f1}')

roc : 0.9999988333938742
f1 : 0.9943524096385542


In [101]:
# After training
model = pipeline.named_steps['model']
preprocessor = pipeline.named_steps['preprocessor']

# Build a production-ready pipeline
from sklearn.pipeline import Pipeline
production_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])


In [102]:
import joblib
joblib.dump(production_pipeline, '..\\Model\\model.pkl')

['..\\Model\\model.pkl']

In [104]:
model = joblib.load('..\\Model\\model.pkl')
y_pred = model.predict(X_test)


In [38]:
X_test.loc[[69028]].to_csv("single_row.csv", index=False)

In [ ]:
import pandas as pd
import mlflow.sklearn
import os, sys

sys.path.append(os.path.abspath('..'))  # adds the parent directory

# Point directly to the model folder under artifacts
model_uri = r"../mlruns/578058596288668658/models/m-c195880d3970492db5671ec190cb6a96/artifacts/"

loaded_model = mlflow.sklearn.load_model(model_uri)

df = pd.read_csv('single_row.csv')

pred = loaded_model.predict(df)
print(pred)
# Use the loaded model
# Example: predictions = loaded_model.predict(X_test)
print("✅ Model loaded successfully:", loaded_model)


[0]
✅ Model loaded successfully: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Amount', 'Value',
                                                   'PricingStrategy',
                                                   'FraudResult',
                                                   'transaction_hour',
                                                   'transaction_day',
                                                   'transaction_month',
                                                   'transaction_year',
                                                   'total_transaction_amount',
                                                   'average_transaction_amount',
                                                   'transaction_count',
                                                   'std_transaction_amount',
                                                   'Recency'